### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import ads

ads.set_auth(auth="resource_principal")

In [2]:
import tensorflow as tf

# Create the TensorFlow Model
mnist = tf.keras.datasets.mnist
(trainx, trainy), (testx, testy) = mnist.load_data()
trainx, testx = trainx / 255.0, testx / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(trainx, trainy, epochs=1)

2025-10-22 22:42:15.131457: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-22 22:42:15.132547: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1875/1875 [==============================] - 6s 3ms/step - loss: 0.2995 - accuracy: 0.9125


In [3]:
# Exercise/validate the prediction
model.predict(testx[0:1])

array([[-0.12721297, -7.258293  ,  2.4165602 ,  2.4989808 , -5.449205  ,
        -2.2823584 , -9.582444  ,  9.480782  , -1.5669403 ,  0.28356847]],
      dtype=float32)

In [6]:
## TENSORFLOW FRAMEWORK SERIALIZATION
# Create an ADS model
from ads.common.model_metadata import UseCaseType
from ads.model.framework.tensorflow_model import TensorFlowModel
from uuid import uuid4

tensorflow_model = TensorFlowModel(estimator=model, artifact_dir=f"./model-artifact-{str(uuid4())}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Prepare the ADS Model
tensorflow_model.prepare(
#    inference_conda_env="tensorflow28_p38_cpu_v1",
#    training_conda_env="tensorflow28_p38_cpu_v1",
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [10]:
# Verify the ADS model
tensorflow_model.verify(testx)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-ea1d71f3-1d7f-410e-83f5-feaf6ec3e4ca ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [11]:
# Save the ADS model
model_id = tensorflow_model.save()

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-ea1d71f3-1d7f-410e-83f5-feaf6ec3e4ca ...
Model is successfully loaded.
['runtime.yaml', 'output_schema.json', 'score.py', 'model.h5']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Available     Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [15]:
# Deploy and create an endpoint for the TensorFlow model
tensorflow_model.deploy(
    display_name="TensorFlow Model For Classification",
    #deployment_log_group_id="ocid1.loggroup.oc1.xxx.xxxxx",
    #deployment_access_log_id="ocid1.log.oc1.xxx.xxxxx",
    #deployment_predict_log_id="ocid1.log.oc1.xxx.xxxxx",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {tensorflow_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-phoenix-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.phx.amaaaaaakoygqqia7odbkohcs6vngoxedxkdm5a4ymv7iofukn2vws56wuha


In [19]:
# Generate prediction by invoking the deployed endpoint
tensorflow_model.predict(testx[:3])['prediction']

[[-0.12721316516399384,
  -7.258293628692627,
  2.4165594577789307,
  2.4989805221557617,
  -5.449204444885254,
  -2.282358407974243,
  -9.582443237304688,
  9.480780601501465,
  -1.5669399499893188,
  0.28356796503067017],
 [0.6263681650161743,
  0.7368243932723999,
  9.652976036071777,
  3.041747570037842,
  -9.405069351196289,
  1.1084423065185547,
  1.8432639837265015,
  -8.964569091796875,
  3.8734588623046875,
  -8.58415699005127],
 [-5.359614849090576,
  4.983343601226807,
  -0.38673123717308044,
  -0.8366215229034424,
  -2.195523500442505,
  -2.277094602584839,
  -2.174736499786377,
  0.2910146117210388,
  -0.8912428617477417,
  -2.195861577987671]]

In [20]:
# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [21]:
# Prepare data sample for prediction and save it to file 'data-payload'
from io import BytesIO
import base64
import numpy as np

data = testx[:3]
np_bytes = BytesIO()
np.save(np_bytes, data, allow_pickle=True)
data = base64.b64encode(np_bytes.getvalue()).decode("utf-8")
with open('data-payload', 'w') as f:
    f.write('{"data": "' + data + '", "data_type": "numpy.ndarray"}')

In [ ]:
# Test the deployment via the OCI CLI with the following shell commands
# $ export uri=https://modeldeployment.{region}.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.xxx.xxxxx/predict
# $ oci raw-request \
#    --http-method POST \
#    --target-uri $uri \
#    --request-body file://data-payload
#
# For this latest deployment this looks like this
# $ export uri=https://modeldeployment.us-phoenix-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.phx.amaaaaaakoygqqia7odbkohcs6vngoxedxkdm5a4ymv7iofukn2vws56wuha/predict
# $ oci raw-request \
#    --http-method POST \
#    --target-uri $uri \
#    --request-body file://data-payload